#### Here is Covered: 
- Simple Agent
- Handoffs - Agent will deligate task to other Agent
- InputGuardrail - Block the result conditionally based on input.


In [1]:
from agents import Agent, Runner, trace,  InputGuardrail, GuardrailFunctionOutput
from agents.exceptions import InputGuardrailTripwireTriggered
from dotenv import load_dotenv
import asyncio
from datetime import datetime
from IPython.display import display, Markdown
from pydantic import BaseModel

In [2]:
load_dotenv()

True

In [ ]:
# Simple Agent Run Process

agent = Agent(
    name="Math Tutor", 
    instructions="You will act as a math tutor. Explain your reasoning step by step and provide the final answer at the end.",
)
result = await Runner.run(agent, "Hello")
result.final_output

HomeworkOutput(is_homework=True, reasoning='The user asked me to act as a math tutor, which implies they are likely seeking help with an educational task, most probably a homework assignment.')

In [5]:
# Homework Guardrail Agent
class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
)
async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered= not final_output.is_homework
    )

In [6]:
#Political Guardrail Agent
class PoliticalOutput(BaseModel):
    is_political: bool
    reasoning: str

political_guardrail_agent = Agent(
    name="Political Guardrail",
    instructions="Check if the user is asking about political topics.",
    output_type=PoliticalOutput,
)
async def political_guardrail(ctx, agent, input_data):
    result = await Runner.run(political_guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(PoliticalOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered= final_output.is_political
    )


In [7]:
# Handoff - Delegate to other agents based on user question
history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Special Agent for historical questions",
    instructions="You will act like a history tutor. Explain important event and context clearly and concisely.",
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Special Agent for math questions",
    instructions="ou will act as a math tutor. Explain your reasoning step by step and provide the final answer at the end."
)

agent_executor = Agent(
    name="Agent Executor",
    instructions="You determine which agent to use based on user question ",
    handoffs=[history_tutor_agent, math_tutor_agent],
    input_guardrails=[
        # InputGuardrail(guardrail_function=homework_guardrail),
        InputGuardrail(guardrail_function=political_guardrail),
    ],
)

In [9]:
with trace("Test handoffs"):
    try:
        result = await Runner.run(agent_executor, "Do you know about me?")
        print(result.final_output)
    except InputGuardrailTripwireTriggered as e:
        print("Guardrail blocked this input:", e)

I don't have information about personal details unless you've shared something specific in this conversation. How can I assist you today?
